In [2]:
from alinea.adel.adel_dynamic import AdelDyn
from openalea.plantgl.all import Viewer, Scene
from alinea.adel.Stand import AgronomicStand
from openalea.mtg import MTG
import os
from alinea.adel.geometric_elements import Leaves
from alinea.adel.newmtg import internode_elements, sheath_elements, blade_elements, convert, properties_from_dict, adel_metamer
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def insert_base_top_element(g):
    ele_level = 5
    ele_nodes_vid = g.vertices(ele_level)
    for e_vid in ele_nodes_vid:
        label = g.node(e_vid).label
        if label.startswith('Hidden'): # hidden element
            base_ele_id = g.insert_parent(e_vid, label='baseElement')
            g.add_component(complex_id=g.node(e_vid).complex()._vid, component_id=base_ele_id, edge_type='/', label='baseElement')
            g.property('edge_type')[e_vid] = '<' # update the existed node's edge_type to '<' for proper display
            if g.node(base_ele_id).parent() is not None: # further inser topElement inbetween its parent
                for n in g.node(base_ele_id).parent().children():
                    if 'topElement' == n.label: # if topElement has already been inserted, use the exsited one.
                        g.insert_parent(base_ele_id, n._vid, label='topElement')
                        break
                else: # not found topElement
                    top_ele_id = g.insert_parent(base_ele_id, label='topElement')
                    g.add_component(complex_id=g.node(base_ele_id).parent().complex()._vid, component_id=top_ele_id, edge_type='<', label='topElement')
               
        elif label.startswith('Leaf'):# leaf element
            g.add_child(e_vid, label='topElement', edge_type='<')
    return g

def label_alter_function(gg):
    """
    change the label name from 'pedoncule' (french) to 'peduncle' (english)
    """
    alter_list = [ vid for vid, label_name in gg.properties()['label'].items() if label_name=='pedoncule']
    for vid in alter_list:
        gg.node(vid).label = 'peduncle'
        
    return gg

In [4]:
PLANT_DENSITY = {1: 200.}
INTER_ROW = 0.15
stand = AgronomicStand(sowing_density=PLANT_DENSITY[1], plant_density=PLANT_DENSITY[1], inter_row=INTER_ROW, noise=0.)
adel_wheat = AdelDyn(nplants=1, duplicate=5, seed=1234, scene_unit='m', stand=stand, aspect='square')
g = adel_wheat.load(dir='./inputs')
g.display()

MTG : nb_vertices=775, nb_scales=6
/plant1			(id=1)											
^/MS			(id=2)											
^/metamer0			(id=3)											
^/collar			(id=4)											
^/baseElement			(id=5)											
^<topElement			(id=6)											
^<metamer1			(id=7)											
^/internode			(id=8)											
^/baseElement			(id=9)											
	+b'T'			(id=181)										
	^/metamer0			(id=182)										
	^/collar			(id=183)										
	^/baseElement			(id=184)										
		+b'T'			(id=431)									
		^/metamer0			(id=432)									
		^/collar			(id=433)									
		^/baseElement			(id=434)									
		^<topElement			(id=435)									
		^<metamer1			(id=436)									
		^/internode			(id=437)									
		^/baseElement			(id=438)									
		^<HiddenElement			(id=447)									
		^<StemElement			(id=446)									
		^<topElement			(id=439)									
			+sheath			(id=440)								
			^/baseElement			(id=441)								
			^<HiddenElement			(id=449)								
			^<StemElement			(id=448)								
			^<topElement			(id=442)								
			^<blade